# Check Dataset
## Pickleデータ

## Pickleデータを格納しているDataFrameを可視化する

In [1]:
import pickle
import pandas as pd
import sunpy.map
import glob
import utils
from tqdm import tqdm
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)
def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [3]:

path = "../coord_dfs/201006coord_df.pickle"
coord_df = pickle_load(path)

In [4]:
from datetime import datetime
coord_df

,Polygon,C_FLARE,M_FLARE,X_FLARE
2010-06-01 00:00:00,"[[(3257.0, 2804.0), (3257.0, 2805.0), (3256.0,...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2010-06-01 01:00:00,"[[(3270.0, 2803.0), (3270.0, 2804.0), (3269.0,...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2010-06-01 02:00:00,"[[(3284.0, 2804.0), (3284.0, 2805.0), (3283.0,...","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
2010-06-01 03:00:00,"[[(3297.0, 2803.0), (3297.0, 2804.0), (3296.0,...","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2010-06-01 04:00:00,"[[(3308.0, 2805.0), (3308.0, 2806.0), (3307.0,...","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...
2010-06-30 20:00:00,"[[(3124.0, 2476.0), (3124.0, 2477.0), (3123.0,...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
2010-06-30 21:00:00,"[[(3143.0, 2476.0), (3143.0, 2477.0), (3142.0,...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
2010-06-30 22:00:00,"[[(3159.0, 2476.0), (3159.0, 2477.0), (3158.0,...","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
2010-06-30 23:00:00,"[[(3169.0, 2477.0), (3169.0, 2478.0), (3168.0,...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"


In [5]:
# 書き出し
coord_df.to_csv("201006.csv")

## データ数のカウント
データのインバランスが学習に与える可能性があるのでデータ内のフレアラベルがTrueのデータとFalseのデータを調べる


In [5]:
df_paths = sorted(glob.glob("../coord_dfs/*coord_df.pickle"))
keys = ["C_FLARE","M_FLARE","X_FLARE"]
for df_path in df_paths:
    coord_df = pickle_load(df_path)
    all_data = 0
    flare_data = 0
    month = df_path[-21:-15]
    print(month)
    for index,row in coord_df.iterrows():
        is_flared = [False for i in range(len(row["Polygon"]))]
        for i,cell in enumerate(is_flared):
            for key in keys:
                all_data +=1
                if row[key][i]!=0:
                    is_flared[i] = True
                    flare_data +=1
    print("all_data:{},flare_data:{}".format(all_data,flare_data))
    with open("imbalance.txt", mode="a") as f:
        f.write("{}:all_data:{},flare_data:{} imbalance:{}\n".format(month,all_data,flare_data,flare_data/all_data))



201005
all_data:10002,flare_data:0
201006
all_data:8628,flare_data:96
201007
all_data:9846,flare_data:55
201008
all_data:12753,flare_data:31
201009
all_data:12336,flare_data:22
201010
all_data:10011,flare_data:54
201011
all_data:8199,flare_data:87
201012
all_data:12027,flare_data:0
201101
all_data:8862,flare_data:0
201102
all_data:12795,flare_data:287
201103
all_data:16197,flare_data:509
201104
all_data:18204,flare_data:408
201105
all_data:23997,flare_data:128
201106
all_data:16983,flare_data:42
201107
all_data:19146,flare_data:364
201108
all_data:23904,flare_data:377
201109
all_data:22044,flare_data:656
201110
all_data:28266,flare_data:525
201111
all_data:27813,flare_data:1004
201112
all_data:31026,flare_data:717
201201
all_data:25800,flare_data:152
201202
all_data:16803,flare_data:54
201203
all_data:22863,flare_data:612
201204
all_data:25128,flare_data:511
201205
all_data:31068,flare_data:394
201206
all_data:23085,flare_data:554
201207
all_data:16296,flare_data:933
201208
all_data:26

ZeroDivisionError: division by zero

In [7]:
import json
with open ("dataset.json", "w") as f:
    json.dump(coco_pickles,f)

NameError: name 'coco_pickles' is not defined

## 改めてデータセットをチェックする(2020/11/16)
そもそも回転しなくてもいい可能性がある
→全部が同じだけずれているのであればそのままで重ねてみればいいかもしれない
M_45s→179.929642
M_720s→
mharp.bitmap→180.0825



In [80]:
full_disk_path = "/Volumes/NO NAME/datasets/HMI_REGION/hmi.M_45s.20100501_000000_TAI.2.magnetogram.fits"
full_disk_map = sunpy.map.Map(full_disk_path)
# full_disk_meta = ["{}:{}".format(key,value) for key,value in full_disk_map.meta.items()]
full_disk_map.meta["crota2"]

179.929642

In [82]:
bitmap_path = "/Volumes/NO NAME/datasets/bitmap/hmi.Mharp_720s.1.20100501_000000_TAI.bitmap.fits"
bitmap_map = sunpy.map.Map(bitmap_path)
bitmap_map.meta["crota2"]
bitmap_meta = ["{}:{}".format(key,value) for key,value in bitmap_map.meta.items()]
bitmap_meta

['simple:True',
 'bitpix:16',
 'naxis:2',
 'naxis1:207',
 'naxis2:279',
 'extend:True',
 "comment:FITS (Flexible Image Transport System) format is defined in 'Astronomy  and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H",
 'date:2012-04-10T05:26:33',
 'telescop:SDO/HMI',
 'instrume:HMI_SIDE1',
 'wavelnth:6173.0',
 'camera:1',
 'bunit:none',
 'content:HMI Active Region Patch',
 'bld_vers:V6R1X',
 'quality:66560',
 'quallev1:0',
 't_obs:2010.05.01_00:00:04_TAI',
 't_rec:2010.05.01_00:00:00_TAI',
 'trecepoc:1993.01.01_00:00:00_TAI',
 'trecstep:720.0',
 'trecunit:secs',
 'cadence:720.0',
 'datasign:1',
 'harpnum:1',
 'date-obs:2010-04-30T23:58:22.20',
 'hflid:1001',
 'hcftid:11',
 'qlook:0',
 'tintnum:8',
 'sintnum:10',
 'distcoef:/home/jsoc/cvs/Development/JSOC/proj/lev1.5_hmi/apps/../libs/lev15/',
 'rotcoef:/home/jsoc/cvs/Development/JSOC/proj/lev1.5_hmi/apps/../libs/lev15/',
 'polcalm:1',
 'codever0:$Id: HMI_IQUV_averaging.c,v 1.19 2011/03/09 18:24:30 couvidat Exp $'

In [13]:
full_disk_path = "/Volumes/NO NAME/datasets/HMI_REGION/hmi.M_720s.20100501_000000_TAI.1.magnetogram.fits"
full_disk_map = sunpy.map.Map(full_disk_path)
full_disk_map.meta["crota2"]

180.0134

In [17]:
sharp_path = "/Volumes/NO NAME/datasets/SHARP(CEA)/JSOC_20201116_517/hmi.sharp_cea_720s.1.20100501_000000_TAI.bitmap.fits"
sharp_map = sunpy.map.Map(sharp_path)
sharp_map.meta["crota2"]

0.0

In [21]:
# 画像書き出し
import cv2
cv2.imwrite("2010050100.png",full_disk_map.data)

True

In [124]:
from rasterio.features import shapes
import numpy as np
ar_polygon = shapes(bitmap_map.data.astype("int16"),mask=None,connectivity = 8)

In [127]:
from shapely.geometry import Polygon
image = cv2.imread("2010050100.png")
overlay=image.copy()
ll_x = bitmap_map.meta["crpix1"]
ll_y = bitmap_map.meta["crpix2"]
int_coords = lambda x:np.array(x).round().astype(np.int32)
alpha = 0.5
for p,j in ar_polygon:
    poly = [(coord[0]+ll_x,coord[1]+ll_y) for coord in p["coordinates"][0]]
    polygon = Polygon(poly)
    exterior = [int_coords(polygon.exterior.coords)]
    cv2.polylines(overlay,exterior,color=(255,255,0),isClosed=True,thickness = 10)
    cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
cv2.imwrite("2010050100_overray.png",image)
    

True